In [1]:
import pandas as pd
df = pd.read_csv("/config/workspace/Diamond_price_prediction/notebook/data/gemstone.csv")

ModuleNotFoundError: No module named 'pandas'

In [13]:
df = df.drop(labels=["id"],axis=1)

In [14]:
# independent feature

X = df.drop(labels=['price'],axis=1)
Y = df[["price"]]

In [15]:
categorial_cols = X.select_dtypes(include="object").columns
numerical_cols = X.select_dtypes(exclude='object').columns

In [16]:
cut_category = ['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
color_cateogry = ['F', 'J', 'G', 'E', 'D', 'H', 'I']
clarity_category = ['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']

In [17]:
from sklearn.impute import SimpleImputer  #this will fill the NA values inside the dataset using mean median mode
from sklearn.preprocessing import StandardScaler  #handling feature scaling
from sklearn.preprocessing import OrdinalEncoder # performing ordinal encoding whenever the categorial feature is applied to rank we use Ordinalencoder
# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [18]:
num_pipeline = Pipeline(
    steps=[
    ("imputer", SimpleImputer(strategy='median')),
    ("scaler",StandardScaler())
    ]
)


#categorial pipeline
cat_pipeline=Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder', OrdinalEncoder(categories=[cut_category,color_cateogry,clarity_category])),
    ('scaler', StandardScaler())

    ]
)
preprocessor = ColumnTransformer([
("num_pipeline",num_pipeline,numerical_cols),
('cat_pipeline',cat_pipeline,categorial_cols)
])

In [19]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.30, random_state=30)

In [20]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())


In [21]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error, mean_squared_error

In [22]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [23]:
import numpy as np 
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_score

In [24]:
models = {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}

model_list =  []
r2_list = []


for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)


    #make prediction
    y_pred = model.predict(X_test)
    mae, rmse, r2_square = evaluate_model(y_test, y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
   

    print('Model training Performance')
    print('RMSE:',rmse)
    print('MAE:' , mae)
    print("R2 score", r2_square)

    r2_list.append(r2_square)


LinearRegression
Model training Performance
RMSE: 1158.7622598854564
MAE: 698.1898929809182
R2 score <function r2_score at 0x7f2bd4be3d30>
Lasso
Model training Performance
RMSE: 1162.672214124898
MAE: 700.6975369014375
R2 score <function r2_score at 0x7f2bd4be3d30>
Ridge
Model training Performance
RMSE: 1160.9269714740462
MAE: 700.2365085027432
R2 score <function r2_score at 0x7f2bd4be3d30>
ElasticNet
Model training Performance
RMSE: 1664.5853812068283
MAE: 1131.576668698669
R2 score <function r2_score at 0x7f2bd4be3d30>


In [25]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']